In [ ]:
###
# App Name:  Converting BIMJSON files to CSV
# App URI: https://opendatabim.io/
# OpenDataBIM
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
###

import os

# Path to files where BIMISON files are located
datajson = []
path = '/content/drive/MyDrive/BIMJSON/26112021_2'
for filename in os.listdir(path):
    if filename.endswith("json"): 
        print(filename)
        datajson.append(filename)

ID-White BIMJSON Test-R21_rvt.json
ID-White BIMJSON Test-R21_ifc.json
['ID-White BIMJSON Test-R21_rvt.json', 'ID-White BIMJSON Test-R21_ifc.json']
--- 0.3022763729095459 seconds ---


In [ ]:
import json
import pandas as pd

# The function of converting each file from the folder that was specified earlier
alldf = []
for file in datajson:
  fileObject = open(path+'/'+file, "r", encoding="utf8", errors='ignore')
  jsonContent = fileObject.read()
  to_python = json.loads(jsonContent)

  # Creating dictionaries for element IDs and their properties
  vocabulary = to_python['data']['objects'][0]['prop'][0]['prop']
  properties = to_python['data']['objects'][0]['prop'][1]['prop']
  vocnames = []
  for x in vocabulary:
    name = x['name']
    vocnames.append(name)
  mass = []
  for x in properties:
    catname = x['name']
    index = vocnames.index(catname)
    propervoc = vocabulary[index]['prop']

    # Getting parameter names and their values
    for y in x['prop']:
      mass2 = []
      id = y['ID']
      key2 = list(y)[1]
      items = y[key2]
      mass2.append(id)
      onlyvalinvoc = []
      onlyvalinvoc2 = []
      for el in propervoc:
        data2 = list(el.items())
        for elem in data2:
          val = elem[1]
          propname = elem[0]
          onlyvalinvoc.append(val)
          onlyvalinvoc2.append(propname)

      # Moving values to a list
      for elements in items:
        allelements = []
        letter_list = elements.split(",")
        allelements.append(onlyvalinvoc2[int(letter_list[0])])
        allelements.append(onlyvalinvoc[int(letter_list[0])][int(letter_list[1])])
        mass2.append(allelements)
      mass.append(mass2)
  
  # Connecting parameters and their values
  allnewarrdict = []
  for el in mass:
      newarrdict = []
      dict_data = {}
      for unel in el[1:]:
          key = [unel[0]]
          val = [unel[1]]
          res = dict(zip(key,val))
          dict_data.update(res)
      newarrdict.append(el[0])
      newarrdict.append(dict_data)
      allnewarrdict.append(newarrdict)     
  print(allnewarrdict)

  # Transferring data to a DataFrame
  dfbig = pd.DataFrame()
  dfbig = dfbig[0:0]

  #Saving data to a file in the same folder with the CSV extension
  for el in allnewarrdict:
    onlyfroms = el[1:][0]
    ids = []
    ids.append(el[0])
    df = pd.DataFrame([onlyfroms], index = ids, columns=onlyfroms.keys())
    dfbig = dfbig.append(df)
  dfbig.to_csv(path + '/' + file + '.csv')
  dfbig.to_excel(path + '/' + file + '.xlsx')
